In [3]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%matplotlib notebook

from threeML import *

In [2]:
data_dir = os.path.join('gbm','bn080916009')
trigger_number = 'bn080916009'

# Download the data

data_dir_gbm = os.path.join('gbm',trigger_number)
gbm_data = download_GBM_trigger_data(trigger_number,detectors=['n3','b0'],destination_directory=data_dir_gbm,compress_tte=True)



src_selection = "0.-5."


nai3 = FermiGBMTTELike('NAI3',
                         os.path.join(data_dir, "glg_tte_n3_bn080916009_v01.fit.gz"),
                         "-10-0,100-200",
                         src_selection,
                         rsp_file=os.path.join(data_dir, "glg_cspec_n3_bn080916009_v00.rsp2"))

bgo0 = FermiGBMTTELike('BGO0',
                         os.path.join(data_dir, "glg_tte_b0_bn080916009_v01.fit.gz"),
                         "-10-0,100-200",
                         src_selection,
                         rsp_file=os.path.join(data_dir, "glg_cspec_b0_bn080916009_v00.rsp2"))


nai3.set_active_measurements("8.0-30.0", "40.0-950.0")
bgo0.set_active_measurements("250-43000")


Auto-determined polynomial order: 1





WARNING UserWarning: No TLMIN keyword found. This DRM does not follow OGIP standards. Assuming TLMIN=1


WARNING VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future



Auto-probed noise models:
- observation: poisson
- background: gaussian
Auto-determined polynomial order: 1


Auto-probed noise models:
- observation: poisson
- background: gaussian
Range 8.0-30.0 translates to channels 4-21
Range 40.0-950.0 translates to channels 27-125
Now using 117 channels out of 128
Range 250-43000 translates to channels 1-126
Now using 126 channels out of 128


In [4]:
nai3.create_time_bins(start=0,stop=10,dt=2,method='constant')
bgo0.read_bins(nai3)


nai3_intervals = nai3.get_ogip_from_binner()
bgo0_intervals = bgo0.get_ogip_from_binner()


all_data = nai3_intervals + bgo0_intervals

In [5]:
triggerName = 'bn080916009'
ra = 121.8
dec = -61.3




data_list = DataList( *all_data )



band = Band()


GRB = PointSource( triggerName, ra, dec, spectral_shape=band  )

model = Model( GRB )


time = IndependentVariable("time",13.0, unit='s')
model

law = Powerlaw(K=500,index=-1)

#law = Line(a=-0.02,b=-2.0)
model.add_independent_variable(time)
model.link(model.bn080916009.spectrum.main.Band.xp,time,law)

mean_time = [np.mean([x,y]) for x,y in zip(nai3.bins[0],nai3.bins[1])]





ff = step_generator(mean_time,band.alpha)



model.link(model.bn080916009.spectrum.main.Band.alpha,time,ff)


ff = step_generator(mean_time,band.K)



model.link(model.bn080916009.spectrum.main.Band.K,time,ff)


ff = step_generator(mean_time,band.beta)



model.link(model.bn080916009.spectrum.main.Band.beta,time,ff)




for n,b,t in zip(nai3_intervals,bgo0_intervals,mean_time):
    
    
    n.external_property(time,t)
    b.external_property(time,t)
    



In [6]:
model

name,value,min_value,max_value,unit,delta,free
bn080916009.spectrum.main.Band.K.composite.value_1,0.0001,None,None,1 / (cm2 keV s),0.1,True
bn080916009.spectrum.main.Band.K.composite.value_2,0.0001,None,None,1 / (cm2 keV s),0.1,True
bn080916009.spectrum.main.Band.K.composite.value_3,0.0001,None,None,1 / (cm2 keV s),0.1,True
bn080916009.spectrum.main.Band.K.composite.value_4,0.0001,None,None,1 / (cm2 keV s),0.1,True
bn080916009.spectrum.main.Band.K.composite.value_5,0.0001,None,None,1 / (cm2 keV s),0.1,True
bn080916009.spectrum.main.Band.alpha.composite.value_1,-1.0,-1.5,3.0,,0.1,True
bn080916009.spectrum.main.Band.alpha.composite.value_2,-1.0,-1.5,3.0,,0.1,True
bn080916009.spectrum.main.Band.alpha.composite.value_3,-1.0,-1.5,3.0,,0.1,True
bn080916009.spectrum.main.Band.alpha.composite.value_4,-1.0,-1.5,3.0,,0.1,True
bn080916009.spectrum.main.Band.alpha.composite.value_5,-1.0,-1.5,3.0,,0.1,True


In [7]:
jl = JointLikelihood( model, data_list, verbose=False )
#jl.set_minimizer("PYOPT","COBYLA")
res = jl.fit()


WARNING RuntimeWarning: divide by zero encountered in log


WARNING RuntimeWarning: invalid value encountered in log


WARNING RuntimeWarning: divide by zero encountered in log


WARNING RuntimeWarning: invalid value encountered in log


WARNING RuntimeWarning: divide by zero encountered in log


WARNING RuntimeWarning: invalid value encountered in log


WARNING RuntimeWarning: invalid value encountered in multiply


WARNING RuntimeWarning: divide by zero encountered in log


WARNING RuntimeWarning: invalid value encountered in log


WARNING RuntimeWarning: invalid value encountered in multiply


WARNING NotANumberInLikelihood: These parameters returned a logLike = Nan: [  2.07288868e+01  -9.56086297e+00   1.26117679e+01   9.43338201e-01
  -1.67599809e+01   2.82404089e+00   2.50709106e+00   2.68650639e+00
   2.07442537e+00   2.21922370e+00   1.73282619e+03   9.17849568e+00
  -2.18749440e+00  -2.55957053e+00  -2.10240814e+00  -3.44166489e+00
  -2.37720318e+00]


WARNING RuntimeWarning:

Best fit values:



#,Name,Best fit value,Unit
0,bn080916009.spectrum.main.Band.K.composite.value_1,0.0183 +/- 0.0006,1 / (cm2 keV s)
1,bn080916009.spectrum.main.Band.K.composite.value_2,0.0231 +/- 0.0006,1 / (cm2 keV s)
2,bn080916009.spectrum.main.Band.K.composite.value_3,0.0214 +/- 0.0006,1 / (cm2 keV s)
3,bn080916009.spectrum.main.Band.K.composite.value_4,0.0158 +/- 0.0005,1 / (cm2 keV s)
4,bn080916009.spectrum.main.Band.K.composite.value_5,0.0092 +/- 0.0005,1 / (cm2 keV s)
5,bn080916009.spectrum.main.Band.alpha.composite.value_1,-1.206 +/- 0.032,
6,bn080916009.spectrum.main.Band.alpha.composite.value_2,-1.266 +/- 0.026,
7,bn080916009.spectrum.main.Band.alpha.composite.value_3,-1.295 +/- 0.027,
8,bn080916009.spectrum.main.Band.alpha.composite.value_4,-1.5000 +/- 0.0014,
9,bn080916009.spectrum.main.Band.alpha.composite.value_5,-1.50000 +/- 0.00006,



NOTE: errors on parameters are approximate. Use get_errors().


Correlation matrix:



1.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00
0.00,1.00,0.14,0.07,0.06,0.00,0.20,0.09,0.01,0.00,-0.00,-0.42,0.00,-0.00,0.17,-0.00,-0.00
0.00,0.14,1.00,0.06,0.05,0.00,0.14,0.15,0.01,0.00,0.00,-0.34,0.00,0.00,-0.03,-0.00,-0.00
0.00,0.07,0.06,1.00,0.03,0.00,0.08,0.04,0.06,0.00,0.00,-0.18,0.00,0.00,0.07,-0.00,-0.00
0.00,0.06,0.05,0.03,1.00,0.00,0.06,0.03,0.00,0.00,0.00,-0.15,0.00,0.00,0.06,-0.00,-0.00
0.05,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00
0.00,0.20,0.14,0.08,0.06,0.00,1.00,0.10,0.01,0.00,-0.00,-0.43,0.00,-0.00,0.18,-0.00,-0.00
0.00,0.09,0.15,0.04,0.03,0.00,0.10,1.00,0.00,0.00,0.00,-0.22,0.00,0.00,-0.19,-0.00,-0.00
0.00,0.01,0.01,0.06,0.00,0.00,0.01,0.00,1.00,0.00,0.00,-0.02,0.00,0.00,0.01,-0.00,-0.00
0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,1.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00



Values of -log(likelihood) at the minimum:



,-log(likelihood)
total,6962.245441
NAI3_0,662.168003
NAI3_1,677.478685
NAI3_2,622.243146
NAI3_3,608.365103
NAI3_4,558.358810
BGO0_0,790.470529
BGO0_1,864.558210
BGO0_2,851.910814
BGO0_3,703.998014


In [ ]:
cleanup_downloaded_GBM_data(gbm_data)